Install module

In [1]:
!pip install transformers tensorflow torch numpy pandas scikit-learn matplotlib tensorflow_hub rouge-score sacrebleu

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 1.9 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=e198f22b255676ed7ed85695447c2ef3c64bb5ac4a14e11e094a57421a68c74f
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


Import modules

In [ ]:
import pandas as pd
import numpy as np
import tensorflow_hub as hub
from sklearn.model_selection import train_test_split
from transformers import T5ForConditionalGeneration, T5Tokenizer, AdamW
import torch
import matplotlib.pyplot as plt
from rouge_score import rouge_scorer
import sacrebleu

Load dataset

In [ ]:
df = pd.read_csv('persona_dataset.csv')
df.dropna(inplace=True)

print(df.head())

Split dataset

In [ ]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

Load USE (Universal Sentence Encoder) from TensorFlow Hub

In [ ]:
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
encoded_train_questions = embed(train_df['question'].tolist())
encoded_test_questions = embed(test_df['question'].tolist())

Load T5 (Text-to-Text Transfer Transformer) Model

In [ ]:
model_name = "t5-small" # or base
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

Train the model with tuned parameters

In [ ]:
# Model training
optimizer = AdamW(model.parameters(), lr=5e-5) # learning rate
model.train()

def train_model(train_df, epochs=3):
    losses = []
    for epoch in range(epochs):
        total_loss = 0
        for _, row in train_df.iterrows():
            input_text = f"question: {row['question']} </s>"
            target_text = f"{row['answer']} </s>"

            input_ids = tokenizer.encode(input_text, return_tensors="pt")
            target_ids = tokenizer.encode(target_text, return_tensors="pt")

            optimizer.zero_grad()
            outputs = model(input_ids=input_ids, labels=target_ids)
            loss = outputs.loss
            total_loss += loss.item()
            loss.backward()
            optimizer.step()

        avg_loss = total_loss / len(train_df)
        losses.append(avg_loss)
        print(f"Epoch {epoch+1}/{epochs} done, Loss average: {avg_loss:.4f}")

    return losses

# Train the model
losses = train_model(train_df)

Loss visualization

In [ ]:
plt.plot(losses, marker='o')
plt.title('Loss visualization when training')
plt.xlabel('Epoch')
plt.ylabel('Loss average')
plt.show()

Model Evaluation and prediction test

In [ ]:
model.eval()
predictions = []

for _, row in test_df.iterrows():
    input_text = f"question: {row['question']} </s>"
    input_ids = tokenizer.encode(input_text, return_tensors="pt")
    outputs = model.generate(input_ids, max_length=50)
    generated_answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    predictions.append(generated_answer)

ROUGE evaluation metrics

In [ ]:
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
rouge_scores = []

for true, pred in zip(test_df['answer'], predictions):
    score = scorer.score(true, pred)
    rouge_scores.append(score)

# Menampilkan rata-rata skor ROUGE
avg_rouge1 = np.mean([score['rouge1'].fmeasure for score in rouge_scores])
avg_rouge2 = np.mean([score['rouge2'].fmeasure for score in rouge_scores])
avg_rougeL = np.mean([score['rougeL'].fmeasure for score in rouge_scores])

print(f"ROUGE-1: {avg_rouge1:.4f}")
print(f"ROUGE-2: {avg_rouge2:.4f}")
print(f"ROUGE-L: {avg_rougeL:.4f}")

BLEU evaluation metrics

In [ ]:
bleu_score = sacrebleu.corpus_bleu(predictions, [test_df['answer'].tolist()])
print(f"BLEU Score: {bleu_score.score:.4f}")

Prediction test

In [ ]:
for i in range(5):
    print("Pertanyaan:", test_df.iloc[i]['question'])
    print("Jawaban Asli:", test_df.iloc[i]['answer'])
    print("Jawaban Prediksi:", predictions[i])
    print()